In [1]:
import csv
import math
import numpy as np
import pandas as pd
libs = pd.read_csv("../export/Lib.csv", ).set_index("groupArtifactId")
apis = pd.read_csv("../export/API.csv", dtype={"signatureId":str}).set_index("signatureId")
libs["signatureIds"] = libs["signatureIds"].apply(lambda x: x.split(";") if type(x) == str else [])
apis["paramList"] = apis["paramList"].apply(lambda x: x.split(";") if type(x) == str else [])

In [2]:
libs.head()

,groupArtifactName,signatureIds
groupArtifactId,,
155,org.junit.jupiter:junit-jupiter-engine,"[1958505103816, 1958505103817, 1958505103818, ..."
61267,commons-discovery:commons-discovery,"[240519190535, 240519190534, 240519190533, 240..."
90,org.assertj:assertj-core,"[1786706395136, 1786706395137, 1786706395138, ..."
83,org.hibernate:hibernate-entitymanager,"[1271310325070, 1271310325071, 2748779075311, ..."
382,org.xerial:sqlite-jdbc,"[2851858334364, 2851858334365, 2851858334366, ..."


In [3]:
apis.head()

,packageName,className,methodName,paramList
signatureId,,,,
2061584302560,com.google.common.collect,ArrayListMultimap,<init>,[]
1305670058288,com.google.common.collect,AbstractMultimap$SortedAsMap,headMap,[java.lang.Object]
3710851744608,com.google.common.collect,Lists$TransformingSequentialList,replaceAll,[java.util.function.UnaryOperator]
2714419331704,com.google.common.base,CharMatcher$6,clone,[]
3195455668968,com.google.common.collect,ForwardingIterator,wait,[]


In [4]:
print(len(set(libs["groupArtifactName"])))
print(set(libs["groupArtifactName"]))

109
{'org.hamcrest:java-hamcrest', 'org.apache.logging.log4j:log4j-api', 'org.hibernate:hibernate-core', 'org.jboss.arquillian.junit:arquillian-junit-container', 'commons-logging:commons-logging', 'org.mockito:mockito-all', 'junit:junit', 'cglib:cglib', 'it.unimi.dsi:fastutil', 'org.reflections:reflections', 'org.sonatype.sisu:sisu-guice', 'com.google.code.gson:gson', 'ch.qos.logback:logback-core', 'org.hsqldb:hsqldb', 'org.easymock:easymock', 'org.eclipse.persistence:eclipselink', 'org.apache.commons:commons-io', 'com.googlecode.json-simple:json-simple', 'javax.inject:javax.inject', 'fastutil:fastutil', 'org.junit.jupiter:junit-jupiter-api', 'org.apache.httpcomponents:httpcore', 'org.slf4j:slf4j-api', 'org.apache.httpcomponents:httpclient', 'commons-lang:commons-lang', 'org.apache.commons:commons-lang3', 'com.google.inject:guice', 'org.hibernate:hibernate-entitymanager', 'jmock:jmock', 'net.sf.json-lib:json-lib', 'org.ow2.asm:asm-commons', 'org.slf4j:slf4j-simple', 'org.slf4j:slf4j-lo

In [5]:
print(libs[libs["signatureIds"].apply(lambda x: len(x) == 0)])

                        groupArtifactName signatureIds
groupArtifactId                                       
284              net.sf.json-lib:json-lib           []


In [6]:
def get_signatures(libname):
    signatureIds = libs[libs["groupArtifactName"] == libname]["signatureIds"].iloc[0]
    return apis.loc[signatureIds]
def to_classes(signatures):
    signatures["packageName"] = signatures["packageName"].apply(lambda x: x if type(x) == str else "")
    return sorted(list(set(signatures["packageName"] + "." + signatures["className"])))
def to_packages(signatures):
    signatures["packageName"] = signatures["packageName"].apply(lambda x: x if type(x) == str else "")
    return sorted(list(set(signatures["packageName"])))
to_classes(get_signatures("org.json:json"))

['org.json.CDL',
 'org.json.Cookie',
 'org.json.CookieList',
 'org.json.HTTP',
 'org.json.HTTPTokener',
 'org.json.JSONArray',
 'org.json.JSONException',
 'org.json.JSONML',
 'org.json.JSONObject',
 'org.json.JSONObject$1',
 'org.json.JSONObject$Null',
 'org.json.JSONPointer',
 'org.json.JSONPointer$Builder',
 'org.json.JSONPointerException',
 'org.json.JSONPropertyIgnore',
 'org.json.JSONPropertyName',
 'org.json.JSONString',
 'org.json.JSONStringer',
 'org.json.JSONTokener',
 'org.json.JSONWriter',
 'org.json.Kim',
 'org.json.Property',
 'org.json.Test',
 'org.json.Test$1Obj',
 'org.json.XML',
 'org.json.XML$1',
 'org.json.XML$1$1',
 'org.json.XMLParserConfiguration',
 'org.json.XMLTokener',
 'org.json.zip.BitInputStream',
 'org.json.zip.BitOutputStream',
 'org.json.zip.BitReader',
 'org.json.zip.BitWriter',
 'org.json.zip.Compressor',
 'org.json.zip.Decompressor',
 'org.json.zip.Huff',
 'org.json.zip.Huff$Symbol',
 'org.json.zip.JSONzip',
 'org.json.zip.Keep',
 'org.json.zip.MapKeep

In [7]:
to_packages(get_signatures("asm:asm"))

['org.objectweb.asm', 'org.objectweb.asm.signature']

In [8]:
# Export to csv for further analysis
import os
from collections import OrderedDict
df = []
libnames = set(libs["groupArtifactName"])
for name in libnames:
    df.append(OrderedDict([
        ["groupArtifactName", name],
        ["hasProblem", ""],
        ["problemDescription", ""],
        ["packages", "\n".join(to_packages(get_signatures(name)))]
    ]))
pd.DataFrame(df).to_csv("lib2pkg.csv", index=False)

In [9]:
get_signatures("net.sf.json-lib:json-lib")

,packageName,className,methodName,paramList
signatureId,,,,


In [11]:
to_classes(get_signatures("org.ow2.asm:asm-commons"))

['.module-info',
 'org.objectweb.asm.commons.AdviceAdapter',
 'org.objectweb.asm.commons.AnalyzerAdapter',
 'org.objectweb.asm.commons.AnnotationRemapper',
 'org.objectweb.asm.commons.ClassRemapper',
 'org.objectweb.asm.commons.CodeSizeEvaluator',
 'org.objectweb.asm.commons.FieldRemapper',
 'org.objectweb.asm.commons.GeneratorAdapter',
 'org.objectweb.asm.commons.InstructionAdapter',
 'org.objectweb.asm.commons.JSRInlinerAdapter',
 'org.objectweb.asm.commons.JSRInlinerAdapter$Instantiation',
 'org.objectweb.asm.commons.LocalVariablesSorter',
 'org.objectweb.asm.commons.Method',
 'org.objectweb.asm.commons.MethodRemapper',
 'org.objectweb.asm.commons.ModuleHashesAttribute',
 'org.objectweb.asm.commons.ModuleRemapper',
 'org.objectweb.asm.commons.ModuleResolutionAttribute',
 'org.objectweb.asm.commons.ModuleTargetAttribute',
 'org.objectweb.asm.commons.RecordComponentRemapper',
 'org.objectweb.asm.commons.Remapper',
 'org.objectweb.asm.commons.RemappingAnnotationAdapter',
 'org.objectwe

In [15]:
to_classes(get_signatures("org.slf4j:slf4j-simple"))

['', 'org.slf4j', 'org.slf4j.impl', 'org.slf4j.simple', 'org.slf4j.spi']

In [17]:
to_packages(get_signatures("org.mockito:mockito-all"))

['net.sf.cglib.asm',
 'net.sf.cglib.asm.attrs',
 'net.sf.cglib.beans',
 'net.sf.cglib.core',
 'net.sf.cglib.proxy',
 'net.sf.cglib.reflect',
 'net.sf.cglib.transform',
 'net.sf.cglib.transform.hook',
 'net.sf.cglib.transform.impl',
 'net.sf.cglib.util',
 'org.hamcrest',
 'org.hamcrest.core',
 'org.hamcrest.internal',
 'org.mockito',
 'org.mockito.asm',
 'org.mockito.asm.signature',
 'org.mockito.asm.tree',
 'org.mockito.asm.tree.analysis',
 'org.mockito.asm.util',
 'org.mockito.cglib.beans',
 'org.mockito.cglib.core',
 'org.mockito.cglib.proxy',
 'org.mockito.cglib.reflect',
 'org.mockito.cglib.transform',
 'org.mockito.cglib.transform.impl',
 'org.mockito.cglib.util',
 'org.mockito.configuration',
 'org.mockito.configuration.experimental',
 'org.mockito.exceptions',
 'org.mockito.exceptions.base',
 'org.mockito.exceptions.cause',
 'org.mockito.exceptions.misusing',
 'org.mockito.exceptions.stacktrace',
 'org.mockito.exceptions.verification',
 'org.mockito.exceptions.verification.junit